## Data Collection

After downloading the movie data from Kaggle, we need more information for our model buiding and future analysis. So we scraped the data from IMDB and Wikipedia.

###  Get the IMDB ID and Average Ratings for Netflix Movies

In [1]:
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('./title.akas.tsv.gz',compression='gzip', sep='\t')


C:\Users\12645\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [17]:
title_id_dict = dict(zip(df.title,df.titleId))

In [3]:
df2 = pd.read_csv('./netflix_all_infomation.csv')

In [10]:
df2 = df2[df2.type=='Movie']

In [11]:
df2.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...


In [15]:
len(df2.title.unique())

6131

In [19]:
id_list = []
for i in df2.title.unique():
    try:
        id_list.append(title_id_dict[i])
    except:
        id_list.append('na')

In [20]:
a = pd.DataFrame({'title':df2.title.unique(), 'id':id_list})

In [21]:
a

,title,id
0,Dick Johnson Is Dead,tt11394180
1,My Little Pony: A New Generation,tt10101702
2,Sankofa,tt8676470
3,The Starling,tt5164438
4,Je Suis Karl,tt9205538
...,...,...
6126,Zinzana,tt4377952
6127,Zodiac,tt7373884
6128,Zombieland,tt9352962
6129,Zoom,tt8870788


In [22]:
df_rating = pd.read_csv('./title.ratings.tsv.gz',compression='gzip', sep='\t')

In [23]:
b = pd.merge(df_rating[df_rating.tconst.apply(lambda x: x in id_list)], a, how='right', left_on='tconst', right_on='id')

In [24]:
c = b[~b.averageRating.isna()]

In [25]:
c

,tconst,averageRating,numVotes,title,id
0,tt11394180,7.4,5864.0,Dick Johnson Is Dead,tt11394180
1,tt10101702,6.9,2685.0,My Little Pony: A New Generation,tt10101702
3,tt5164438,6.3,9055.0,The Starling,tt5164438
4,tt9205538,5.4,608.0,Je Suis Karl,tt9205538
5,tt15204288,5.2,638.0,Confessions of an Invisible Girl,tt15204288
...,...,...,...,...,...
6126,tt4377952,7.1,1602.0,Zinzana,tt4377952
6127,tt7373884,8.3,7.0,Zodiac,tt7373884
6128,tt9352962,5.9,8.0,Zombieland,tt9352962
6129,tt8870788,6.4,7.0,Zoom,tt8870788


After we got the IMDB rating, we also scraped some of the reviews for future use.

###  Get IMDB Reviews for Netflix Movies

In [123]:
def get_review(url):
    import pandas as pd
    
    baseUrl='https://www.imdb.com/title/'+url+'/reviews?ref_=tt_urv'
    response=get(baseUrl)
    html_soup=BeautifulSoup(response.text,'html.parser')
    review_5 = html_soup.find_all('div', class_='text show-more__control')[0:10]

    
    rating_5 = html_soup.find_all('span',class_='rating-other-user-rating')[0:10]
    rating_5 = list(map(lambda x: x.span.text, rating_5))
    
    return pd.DataFrame({'url':[url]*10, 'review':review_5, 'rating':rating_5})

In [124]:
get_review('tt1365050')

,url,review,rating
0,tt1365050,[I have seen several documentaries about the i...,6
1,tt1365050,[Agu is a young boy in an unnamed African nati...,8
2,tt1365050,[Director Cary Fukunaga brings us the turmoil ...,5
3,tt1365050,[Agu (Abraham Attah) is a young lad growing up...,10
4,tt1365050,[You know when you hear other people whining a...,9
5,tt1365050,[Definitely not a film for the timid or sensit...,8
6,tt1365050,[I had found about this film either when I was...,7
7,tt1365050,[Agu is a young boy in a war-torn African coun...,9
8,tt1365050,[I've heard more than one person say that they...,8
9,tt1365050,[Greetings again from the darkness. Cary Joji ...,8


In [125]:
import numpy as np
df_review = pd.DataFrame({'url':[], 'review':[], 'rating':[]})

for x in c.tconst:

    try:
        df_review = pd.concat([df_review, get_review(x)])
    except:
        continue


In [126]:
df_review

,url,review,rating
0,tt1365050,[I have seen several documentaries about the i...,6
1,tt1365050,[Agu is a young boy in an unnamed African nati...,8
2,tt1365050,[Director Cary Fukunaga brings us the turmoil ...,5
3,tt1365050,[Agu (Abraham Attah) is a young lad growing up...,10
4,tt1365050,[You know when you hear other people whining a...,9
...,...,...,...
5,tt15469820,"[In some ways, the fascination with Britney Sp...",6
6,tt15469820,[I gave up half way through..... says it all r...,3
7,tt15469820,[That documentary was amazing! It revealed so ...,9
8,tt15469820,"[In this documentary, despite the fact that th...",8


In [127]:
final_review = pd.merge(c, df_review, left_on='tconst', right_on='url', how='inner')

In [130]:
final_review.to_csv('s3://671happy-yulingai/final_review.csv')

In [129]:
final_review

,tconst,averageRating,numVotes,title,id,url,review,rating
0,tt1365050,7.7,78276.0,Beasts of No Nation,tt1365050,tt1365050,[I have seen several documentaries about the i...,6
1,tt1365050,7.7,78276.0,Beasts of No Nation,tt1365050,tt1365050,[Agu is a young boy in an unnamed African nati...,8
2,tt1365050,7.7,78276.0,Beasts of No Nation,tt1365050,tt1365050,[Director Cary Fukunaga brings us the turmoil ...,5
3,tt1365050,7.7,78276.0,Beasts of No Nation,tt1365050,tt1365050,[Agu (Abraham Attah) is a young lad growing up...,10
4,tt1365050,7.7,78276.0,Beasts of No Nation,tt1365050,tt1365050,[You know when you hear other people whining a...,9
...,...,...,...,...,...,...,...,...
3875,tt15469820,6.4,5442.0,Britney vs Spears,tt15469820,tt15469820,"[In some ways, the fascination with Britney Sp...",6
3876,tt15469820,6.4,5442.0,Britney vs Spears,tt15469820,tt15469820,[I gave up half way through..... says it all r...,3
3877,tt15469820,6.4,5442.0,Britney vs Spears,tt15469820,tt15469820,[That documentary was amazing! It revealed so ...,9
3878,tt15469820,6.4,5442.0,Britney vs Spears,tt15469820,tt15469820,"[In this documentary, despite the fact that th...",8
